In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt
import seaborn as sns
import sqlite3

In [14]:
conection = sqlite3.connect("amazon_sales.db")
df = pd.read_sql_query("""
                        SELECT *
                        FROM products
                        JOIN (
                            SELECT reviews.review_title,reviews.review_content, users.user_name,reviews.id_product,reviews.id_user
                            FROM reviews
                            JOIN users ON reviews.id_user = users.id
                        ) AS reviews_users ON products.id = reviews_users.id_product; 
                        """,conection)

In [15]:
df.head()

,id,product_name,category_1,category_2,category_3,category_4,category_5,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,img_link,product_link,review_title,review_content,user_name,id_product,id_user
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,399.0,1099.0,0.64,4.2,24269,High Compatibility : Compatible With iPhone 12...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,Satisfied,Looks durable Charging is fine tooNo complains,Manav,B07JW9H4J1,AG3D6O4STAQKAY2UVGEUV46KN35Q
1,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,399.0,1099.0,0.64,4.2,24269,High Compatibility : Compatible With iPhone 12...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,Charging is really fast,Charging is really fast,Adarsh gupta,B07JW9H4J1,AHMY5CWJMMK5BJRBBSNLYT3ONILA
2,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,399.0,1099.0,0.64,4.2,24269,High Compatibility : Compatible With iPhone 12...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,Value for money,good product.,Sundeep,B07JW9H4J1,AHCTC6ULH4XB6YHDY6PCH2R772LQ
3,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,399.0,1099.0,0.64,4.2,24269,High Compatibility : Compatible With iPhone 12...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,Product review,Till now satisfied with the quality.,S.Sayeed Ahmed,B07JW9H4J1,AGYHHIERNXKA6P5T7CZLXKVPT7IQ
4,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,399.0,1099.0,0.64,4.2,24269,High Compatibility : Compatible With iPhone 12...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,Good quality,This is a good product . The charging speed is...,jaspreet singh,B07JW9H4J1,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q


In [4]:
df.shape

(2590, 19)

In [5]:
numeric_df = df[["discounted_price",	"actual_price",	"discount_percentage",	"rating",	"rating_count"]]

In [6]:
numeric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2590 entries, 0 to 2589
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   discounted_price     2590 non-null   float64
 1   actual_price         2590 non-null   float64
 2   discount_percentage  2590 non-null   float64
 3   rating               2590 non-null   float64
 4   rating_count         2590 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 101.3 KB


# Handeling Missing Values

In [7]:
df.isna().sum()

id                     0
product_name           0
category_1             0
category_2             0
category_3             0
category_4             0
category_5             0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           0
about_product          0
img_link               0
product_link           0
review_title           0
review_content         0
user_name              0
id_product             0
dtype: int64

# Handeling Duplicates

In [8]:
df[df.duplicated()]

,id,product_name,category_1,category_2,category_3,category_4,category_5,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,img_link,product_link,review_title,review_content,user_name,id_product


# Preguntas Frecuentes 

1. ¿Cuáles son los productos más vendidos?
2. ¿Existen patrones estacionales en las ventas?
3. ¿Qué canales de marketing son más efectivos?
4. ¿Cuál es el valor promedio de un pedido?
5. ¿Qué segmentos de clientes son más rentables?

In [ ]:
products = pd.read_sql_query("""
                        SELECT *
                        FROM products; 
                        """,conection)

In [ ]:
products.head()

,id,product_name,category_1,category_2,category_3,category_4,category_5,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,399.0,1099.0,0.64,4.2,24269,High Compatibility : Compatible With iPhone 12...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,199.0,349.0,0.43,4.0,43994,"Compatible with all Type C enabled devices, be...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,199.0,1899.0,0.90,3.9,7928,? Fast Charger& Data Sync?-With built-in safet...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,329.0,699.0,0.53,4.2,94363,The boAt Deuce USB 300 2 in 1 cable is compati...,https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,154.0,399.0,0.61,4.2,16905,[CHARGE & SYNC FUNCTION]- This cable comes wit...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


Best Seller

In [41]:
maximum_rating = df["rating_count"].max()

best_seller = df.loc[df["rating_count"] == maximum_rating,["product_name","img_link","product_link"]]

print(best_seller["product_name"].unique(),"\n",best_seller["img_link"].unique(),"\n",best_seller["product_link"].unique())

['AmazonBasics Flexible Premium HDMI Cable (Black, 4K@60Hz, 18Gbps), 3-Foot'] 
 ['https://m.media-amazon.com/images/I/41nPYaWA+ML._SY300_SX300_.jpg'] 
 ['https://www.amazon.in/AmazonBasics-Flexible-HDMI-Cable-3-Foot/dp/B07KSMBL2H/ref=sr_1_14?qid=1672909124&s=electronics&sr=1-14']


Segmento mas rentable

In [59]:
most_profitable = pd.read_sql_query(""" 
                        SELECT category_1, category_2, category_3, category_4, category_5, SUM(actual_price) AS total_price
                        FROM products
                        GROUP BY category_1, category_2, category_3, category_4, category_5
                        ORDER BY total_price DESC
                        LIMIT 5;
                        """,conection)

In [60]:
most_profitable

,category_1,category_2,category_3,category_4,category_5,total_price
0,Home&Kitchen,Kitchen&HomeAppliances,"Vacuum,Cleaning&Ironing",Vacuums&FloorCare,Vacuums,216809.00
1,Computers&Accessories,Accessories&Peripherals,Cables&Accessories,Cables,USBCables,150032.66
2,Home&Kitchen,Kitchen&HomeAppliances,"Vacuum,Cleaning&Ironing","Irons,Steamers&Accessories",Irons,77977.00
3,Home&Kitchen,Kitchen&HomeAppliances,SmallKitchenAppliances,DeepFatFryers,AirFryers,60584.00
4,Home&Kitchen,Kitchen&HomeAppliances,SmallKitchenAppliances,Kettles&HotWaterDispensers,ElectricKettles,35087.00
